**2a. Data cleaning: Removing null values**

In [ ]:
# enabling copy on write to avoid creating unnecessary copies
pd.options.mode.copy_on_write = True

In [ ]:
# filling null values in series with unknown
df_goData.fillna('unknown', inplace=True)

In [ ]:
# confirming there are no cells containing null in the dataframe
df_goData.info()

In [ ]:
# confirming there are no cells containing null in the dataframe
df_goData.isnull().sum()

In [ ]:
# checking for duplicate rows
duplicate_rows = df_goData.duplicated()
duplicate_rows

In [ ]:
duplicate_rows.nunique()

**Since there is only 1 unique entry in duplicate rows, it means that there are no duplicates**

In [ ]:
df_goData.shape

### **2c. Data cleaning: Removing Outliers**

**Removing outliers from price coulum**

In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df_goData['price'].quantile(0.25)
Q3 = df_goData['price'].quantile(0.75)

# Calculate IQR
IQR = Q3 - Q1

# Define the bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df_no_outliers = df_goData[(df_goData['price'] >= lower_bound) & (df_goData['price'] <= upper_bound)]




In [ ]:
df_no_outliers.shape

**Removing outliers from mileage column**

In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df_goData['mileage'].quantile(0.25)
Q3 = df_goData['mileage'].quantile(0.75)

# Calculate IQR
IQR = Q3 - Q1

# Define the bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df_no_outliers = df_goData[(df_goData['mileage'] >= lower_bound) & (df_goData['mileage'] <= upper_bound)]

In [ ]:
df_no_outliers.shape

In [ ]:
df_no_outliers.reset_index(drop=True, inplace=True)

In [ ]:
df_no_outliers.info()

## **2d. Data cleaning: Assigning 6 and 7 in transmission_from_vin column to manual and automatic respectively**

In [ ]:
# checking the unique entries for transmission_from_vin column
df_no_outliers['transmission_from_vin'].unique()

In [ ]:
# checking the unique entries for transmission_from_vin column after replacing 6 and 7
df_no_outliers['transmission_from_vin'].unique()

### **3. Identifying Significant Attributes for Problem 3.**

Based on our research into car features, we identified 18 features that has a high potential to make accurate predictions on vehicle transmission type. Following this selction, we are using Chi-square technique to identify features (from these 18) that would best make good predictions, there by reducing the number of features from 18 initially selected.

In [ ]:
# creating a new dataframe containing relevant features
df_features = df_no_outliers[['model_year', 'make', 'model', 'mileage', 'price', 'series', 'style', 'dealer_type', 'stock_type', 'days_on_market', 'certified', 'vin',
                         'drivetrain_from_vin', 'engine_from_vin', 'wheelbase_from_vin','fuel_type_from_vin', 'number_price_changes','transmission_from_vin']]

In [ ]:
df_features.info()

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder # Import OrdinalEncoder

X = df_features.drop('transmission_from_vin', axis=1)  # Features
y = df_features['transmission_from_vin']  # Target

# Convert categorical features to numerical using OrdinalEncoder
encoder = OrdinalEncoder() # Initialize OrdinalEncoder
X_encoded = encoder.fit_transform(X) # Fit and transform X

# Chi-squared test
chi_scores = chi2(X_encoded, y) # Use encoded X for chi2 test
p_values = pd.Series(chi_scores[1], index=X.columns)
p_values.sort_values(ascending=True, inplace=True)
print(p_values)  # Features with lower p-values are more important

From the result of Chi-Square test, **model_year, model, number_price_changes, stock_type, dealer_type, fuel_type_from_vin, and certified** have the lowest p-values and are the most useful in making accurate predictions. In addition to these 7, we will include **make, mileage and price** which we have been instructed to include as features in our model.

In [ ]:
# creating a new dataframe containing relevant 11 features
df_model_features = df_features[['model_year', 'make', 'model', 'mileage', 'price', 'number_price_changes',
                              'stock_type', 'dealer_type', 'fuel_type_from_vin', 'certified', 'transmission_from_vin']]

# displaying the new dataframe
df_model_features.head()

In [ ]:
df_model_features.shape

In [ ]:
df_model_features.info()